In [2]:
from __future__ import print_function, division
#virtualenv -p python3 venv

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [4]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

Using TensorFlow backend.


In [5]:
ls ../data/train/

Type_1/ Type_2/ Type_3/


In [6]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.resize_n, n=299);

In [7]:
train, validate, test = c.data.split_df(df); 

In [8]:
print(train.shape)
train.head()

(888, 5)


,path,Type,filetype,num_id,processed_path
137,../data/train/Type_1/441.jpg,1,jpg,441,../data/processed/resize_299/train/441.jpg
130,../data/train/Type_1/396.jpg,1,jpg,396,../data/processed/resize_299/train/396.jpg
1032,../data/train/Type_3/1003.jpg,3,jpg,1003,../data/processed/resize_299/train/1003.jpg
177,../data/train/Type_1/623.jpg,1,jpg,623,../data/processed/resize_299/train/623.jpg
1303,../data/train/Type_3/480.jpg,3,jpg,480,../data/processed/resize_299/train/480.jpg


In [9]:
print(validate.shape)
validate.head()

(296, 5)


,path,Type,filetype,num_id,processed_path
1126,../data/train/Type_3/1300.jpg,3,jpg,1300,../data/processed/resize_299/train/1300.jpg
1446,../data/train/Type_3/897.jpg,3,jpg,897,../data/processed/resize_299/train/897.jpg
588,../data/train/Type_2/225.jpg,2,jpg,225,../data/processed/resize_299/train/225.jpg
907,../data/train/Type_2/785.jpg,2,jpg,785,../data/processed/resize_299/train/785.jpg
674,../data/train/Type_2/381.jpg,2,jpg,381,../data/processed/resize_299/train/381.jpg


In [10]:
print(test.shape)
test.head()

(297, 5)


,path,Type,filetype,num_id,processed_path
1067,../data/train/Type_3/1107.jpg,3,jpg,1107,../data/processed/resize_299/train/1107.jpg
1305,../data/train/Type_3/49.jpg,3,jpg,49,../data/processed/resize_299/train/49.jpg
328,../data/train/Type_2/1138.jpg,2,jpg,1138,../data/processed/resize_299/train/1138.jpg
704,../data/train/Type_2/428.jpg,2,jpg,428,../data/processed/resize_299/train/428.jpg
191,../data/train/Type_1/7.jpg,1,jpg,7,../data/processed/resize_299/train/7.jpg


# Model creation

In [11]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
    # we have three classes so usung 3 on dense predictions
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# compile the model (should be done *after* setting layers to non-trainable)
# TODO: loss function should be log loss. 'categorical_crossentropy' / c.submission.keras_log_loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv2d_1 False
2 batch_normalization_1 False
3 activation_1 False
4 conv2d_2 False
5 batch_normalization_2 False
6 activation_2 False
7 conv2d_3 False
8 batch_normalization_3 False
9 activation_3 False
10 max_pooling2d_1 False
11 conv2d_4 False
12 batch_normalization_4 False
13 activation_4 False
14 conv2d_5 False
15 batch_normalization_5 False
16 activation_5 False
17 max_pooling2d_2 False
18 conv2d_9 False
19 batch_normalization_9 False
20 activation_9 False
21 conv2d_7 False
22 conv2d_10 False
23 batch_normalization_7 False
24 batch_normalization_10 False
25 activation_7 False
26 activation_10 False
27 average_pooling2d_1 False
28 conv2d_6 False
29 conv2d_8 False
30 conv2d_11 False
31 conv2d_12 False
32 batch_normalization_6 False
33 batch_normalization_8 False
34 batch_normalization_11 False
35 batch_normalization_12 False
36 activation_6 False
37 activation_8 False
38 activation_11 False
39 activation_12 False
40 mixed0 False
41 conv2d_16 False
42 batch_normaliza

In [75]:
# train the model on the new data for a few epochs
batch_size = 20
model.fit_generator(c.processing.df_to_keras_generator(train, batch_size, grayscale=False), 
                    steps_per_epoch=5,
                    epochs=9)

# at this point, the top layers are well trained 
# NEXT STEPS: we can start fine-tuning the lower down convolutional layers from inception V3

Epoch 1/9
[ 1.  0.  0.]
[ 1.  0.  0.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 1.  0.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 1.  0.  0.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  0.  1.]
[ 0.  1.  0.]
[ 1.  0.  0.]
[ 0.  1.  0.]
[ 0.  1.  

KeyboardInterrupt: 

In [72]:
batch_size_t = 10
model.evaluate_generator(c.processing.df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,int(len(test)/batch_size_t))

7.9478884976485684

In [73]:
batch_size_t = 10
model.predict_generator(c.processing.df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,int(len(test)/batch_size_t))

array([[  9.98014558e-16,   1.00000000e+00,   4.31966008e-15],
       [  9.26390631e-11,   1.00000000e+00,   2.29304881e-10],
       [  2.78341044e-13,   1.00000000e+00,   6.54754625e-13],
       [  3.63262406e-11,   1.00000000e+00,   1.03390116e-10],
       [  5.59724732e-11,   1.00000000e+00,   1.24234609e-10],
       [  2.43604824e-11,   1.00000000e+00,   5.60897093e-11],
       [  2.72828260e-10,   1.00000000e+00,   6.46371290e-10],
       [  1.33148971e-12,   1.00000000e+00,   4.78690091e-12],
       [  3.73645986e-14,   1.00000000e+00,   1.76173732e-13],
       [  1.29021016e-14,   1.00000000e+00,   4.69867167e-14],
       [  3.19257945e-12,   1.00000000e+00,   9.06211998e-12],
       [  3.01047139e-11,   1.00000000e+00,   4.53501126e-11],
       [  9.08563589e-13,   1.00000000e+00,   2.55154361e-12],
       [  1.48971303e-13,   1.00000000e+00,   9.96756714e-13],
       [  1.40001871e-10,   1.00000000e+00,   3.10483306e-10],
       [  9.68849770e-14,   1.00000000e+00,   3.7818186

In [ ]:
ldf.sort_values('log_l', ascending=False).path.iloc[0]

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[3])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).resize_100_path.iloc[3])